In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import json

# Load data
file_path = "/kaggle/input/outsystemforums/processed_forum_data.json"

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Check structure
print(f"Total posts: {len(data)}")
print(json.dumps(data[:2], indent=4))  # Show first 2 posts


Total posts: 6882
[
    {
        "title": "ODC Community Tenant - New ODC Components",
        "url": "https://www.outsystems.com/forums/discussion/101643/odc-community-tenant-new-odc-components/",
        "description": "Hi Community,\n\n[Image_1]\n\nIf you have ideas and would like to contribute, fill in your information here: https://forms.gle/u3NcbRpMoDxYPbko9.\n\nAnd if you have any suggestions for components that should be developed on ODC, let us know in this thread!",
        "images": [
            "..."
        ],
        "solutions": [
            {
                "text": "Great news for ODC community!",
                "images": []
            },
            {
                "text": "Wow, thank you to Sebastian, Andrea, and Remco.",
                "images": []
            }
        ]
    },
    {
        "title": "Announcing the OutSystems MVP Award 2024 Winners!",
        "url": "https://www.outsystems.com/forums/discussion/101645/announcing-the-outsystems-mvp-award-20

In [6]:
import json
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import base64
import numpy as np


In [7]:
# Initialize necessary NLP tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [8]:
# Function to clean text (remove punctuation, stopwords, and lemmatize)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(f"[{string.punctuation}]", " ", text)  # Remove punctuation
    text = re.sub(r"\d+", " ", text)  # Remove numbers
    tokens = word_tokenize(text)  # Tokenize the text
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize tokens
    return " ".join(tokens)


In [22]:
import nltk

# Download the necessary resources for NLTK
nltk.download('punkt')  # Tokenizer
nltk.download('stopwords')  # Stopwords
nltk.download('wordnet')  # WordNet for lemmatization

print("✅ NLTK resources downloaded successfully.")


[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
✅ NLTK resources downloaded successfully.


In [ ]:
!pip install textblob


In [1]:
import json
import re
import spacy
import shutil

# Initialize spaCy for NLP
nlp = spacy.load("en_core_web_sm")

# Set test mode to True to process only 500 posts for debugging
TEST_MODE = False

# Function to clean and normalize text (NO SPELL CHECKING for speed)
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()  # Remove unnecessary whitespace
    text = re.sub(r'\d+', '', text)  # Remove numbers if not needed
    text = text.lower()  # Convert text to lowercase
    return text

def process_text_with_images(text, images):
    """Replace [Image_X] with Base64 image data and handle redundant image references"""
    if "[Image_" not in text:  # Skip if no image placeholders
        return text

    image_pattern = re.compile(r"\[Image_(\d+)\]")

    def replace_match(match):
        index = int(match.group(1))
        if index < len(images):
            # If an image exists, replace the placeholder with the actual Base64 data
            return f"[Image: {images[index]}]"
        return match.group(0)  # If no matching image, keep the placeholder

    text = image_pattern.sub(replace_match, text)

    # If the text only contains a Base64 image reference (like "[Image: Base64]"), remove the placeholder
    # Remove unnecessary placeholders when the Base64 data is already in the 'images' field
    for i, img in enumerate(images):
        placeholder = f"[Image_{i}]"
        if placeholder in text:
            text = text.replace(placeholder, "")  # Remove placeholder if the image is already in 'images'

    return text


# Function to process a single forum post
def process_post(post):
    """ Process a forum post by cleaning text and embedding images """
    description_text = post["description"].get("text", "") if isinstance(post["description"], dict) else post["description"]
    description_images = post["description"].get("images", []) if isinstance(post["description"], dict) else []

    description_text = clean_text(description_text)
    description_text = process_text_with_images(description_text, description_images)

    solutions = []
    for solution in post.get("solutions", []):
        solution_text = clean_text(solution["text"])
        solution_text = process_text_with_images(solution_text, solution.get("images", []))
        solutions.append({"text": solution_text, "images": solution.get("images", [])})

    return {
        "title": post["title"],
        "url": post["url"],
        "description": {"text": description_text, "images": description_images},
        "solutions": solutions
    }

# Load the dataset from the uploaded JSON file
file_path = "/kaggle/input/outsystemforums/processed_forum_data.json"  # Adjust path if needed
with open(file_path, 'r', encoding='utf-8') as f:
    forum_posts = json.load(f)

print(f"✅ Loaded {len(forum_posts)} forum posts from the dataset.")

# Process only the first 500 posts if TEST_MODE is enabled
if TEST_MODE:
    forum_posts = forum_posts[:500]  # 🚀 Only process 500 posts for debugging
    print("⚡ Running in TEST MODE: Processing only 500 posts for speed testing.")

# Process the forum posts with logging
processed_posts = []
for idx, post in enumerate(forum_posts):
    processed_post = process_post(post)
    processed_posts.append(processed_post)

    # Print progress every 500 posts
    if (idx + 1) % 500 == 0:
        print(f"✅ Processed {idx + 1}/{len(forum_posts)} posts...")

print(f"✅ Finished processing {len(processed_posts)} posts!")

# Save the processed data to a JSON file
output_file = "/kaggle/working/processed_forum_data_cleaned.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(processed_posts, f, ensure_ascii=False, indent=4)

print(f"✅ Preprocessing complete! Processed data saved to {output_file}")

shutil.make_archive("/kaggle/working/processed_forum_data_cleaned", 'zip', "/kaggle/working", "processed_forum_data_cleaned.json")


✅ Loaded 6882 forum posts from the dataset.
✅ Processed 500/6882 posts...
✅ Processed 1000/6882 posts...
✅ Processed 1500/6882 posts...
✅ Processed 2000/6882 posts...
✅ Processed 2500/6882 posts...
✅ Processed 3000/6882 posts...
✅ Processed 3500/6882 posts...
✅ Processed 4000/6882 posts...
✅ Processed 4500/6882 posts...
✅ Processed 5000/6882 posts...
✅ Processed 5500/6882 posts...
✅ Processed 6000/6882 posts...
✅ Processed 6500/6882 posts...
✅ Finished processing 6882 posts!
✅ Preprocessing complete! Processed data saved to /kaggle/working/processed_forum_data_cleaned.json


'/kaggle/working/processed_forum_data_cleaned.zip'